In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from collections import OrderedDict


#Numerical operations
import numpy as np

# Import the backtrader platform
import backtrader as bt
import backtrader.analyzers as btanalyzers

#Module to analyze portfolio
import pyfolio as pf

#debugging
from ipdb import set_trace

#ploting
import matplotlib.pyplot as plt

#operating system module
import os

#data analysis module
import pandas as pd



class GenericCSVData(bt.feeds.GenericCSVData):
    lines = ('adj_close', 'roe', 'roic', 'roa', 'ebit_ev') 
    params = dict(
        adj_close=5,
        volume=6,
        roe=7,
        roic=8,
        roa=9,
        ebit_ev=10,
        dtformat='%Y-%m-%d',  # fix date format a yyyy-mm-dd
        openinterest=-1,  # -1 indicates there is no openinterest field
        
    )


class gBm(bt.Indicator):
    """
        Geometric Brownian Motion model.
                dS_t = μ S_t dt + σ S_t dB_t
        
        Crescimento Exponencial. 
        o modelo tem um fit melhor para growth stocks como AAPL, TSLA, MGLU3, PRIO3 etc i.e.
        para ativos que tem um Crescimento Exponencial
    """
    lines = ('m', 'σ', 'μ', 'var', 'sl')
    params = dict(size=2**9+1, 
                  sl_quantil=0.1, 
                  sl_look_forward=5)
    
    def __init__(self):
        self.addminperiod(self.p.size)
        
    def next(self):
        self.S = self.data.close.get(0, size=self.p.size)
        self.estimateParams()
        

        self.lines.m[0] = self.m
        self.lines.σ[0] = self.σ
        self.lines.μ[0] = self.μ
        self.lines.var[0] = self.Var(self.p.sl_look_forward)
        self.lines.sl[0] = self.q(self.p.sl_quantil, self.p.sl_look_forward)
        

    
    def estimateParams(self):
        """
        Referência:
        "Estimation of Geometric Brownian Motion Parameters for Oil Price Analysis" C. Jakob et al.
        """
        S = self.S
        X = np.diff(np.log(S), n=1)
        m = X.mean() #mean
        σ = X.std() #standard deviation
        μ = m + ((σ**2)/2) #drift
        n = len(S)

        self.m = m
        self.σ = σ
        self.μ = μ
        self.n = n
    
    def E(self, t):
        """
        Referência:
        Ross, Sheldon M. (2014). "Variations on Brownian Motion".
        Introduction to Probability Models (11th ed.).
        """
        S = self.S
        S0 = S[-1]
        μ = self.μ
        return S0*np.exp(μ*t)
    
    def Var(self, t):
        """
        Referência:
        Ross, Sheldon M. (2014). "Variations on Brownian Motion". 
        Introduction to Probability Models (11th ed.).
        """
        S = self.S
        S0 = S[-1]
        μ = self.μ
        σ = self.σ
        return (S0**2)*np.exp(2*μ*t)*(np.exp((σ**2)*t) - 1)
    
    def q(self, p, t):
        """
         quantil de St/S0 o qual é definido como:
                q(p) = exp( (μ - σ**2/2)*t + σ*np.sqrt(t)*inv_Φ(p))
         p ∈ (0,1)
        """

        σ = self.σ
        μ = self.μ
        
        mean = (μ - (σ**2/2))*t
        var = σ**2*t
        return np.exp(mean +  np.sqrt(var)*N.ppf(p, 0, 1))

class kelly_criterion():
    def __init__(self):
        pass
    
    def gBm(self, μ:list, σ:list, ρ:[], independent=True):
        self.μ = μ
        self.σ = σ
        N = len(μ)
        A = self.A
        B = self.B
        if independent:
            M = np.array([[A(l) if l==l_ else B(l)*B(l_) for l in range(N)] for l_ in range(N)])
        else:
            Mij = np.exp(μ[0]+μ[1]+ρ*σ[0]*σ[1]) = np.exp(μ[0]) - np.exp(μ[1]) + 1 #s.t. i!=j
            M = np.array([[A(l) if l==l_ else  for B in range(N)] for l_ in range(N)])
        b = np.array([B(l) for l in range(N)])
        
        return np.dot(inv(M), b)
    
    def B(self, l):
        μ = self.μ
        return np.exp(μ[l]) - 1
    
    def A(self, l):
        μ = self.μ
        σ = self.σ
        return 1 - 2*np.exp(μ[l]) + np.exp(2*μ[l] + σ[l])
    
class momentumStrat(bt.Strategy):    
    params = dict(
        enableLog=True, 
        gBmSize=2**8+1,
        rebal_monthday=1,
        stocks_in_portfolio=8,
        first_filter='d.roa', 
        second_filter='d.ebit_ev')
        #filtros disponiveis: roa, roic, roe, ebit_ev 
        #obs - sempre coloque o "d." para que funcione coretamente no código
    
    def __init__(self):
        self.order = None
        

        gBms = [gBm(d, size=self.p.gBmSize for d in self.datas]
        driftAcoes = [gBm.μ for gBm in gBms]
        volatAcoes = [gBm.σ for gBm in gBms]

        self.driftAcoes = {d:drift for d, drift in zip(acoes, driftAcoes)}
        self.volatAcoes = {d:volat for d, volat in zip(acoes, volatAcoes)}

        self.add_timer(
            when=bt.Timer.SESSION_START,
            monthdays=[self.p.rebal_monthday],
            monthcarry=True  # if a day isn't there, execute on the next
        )

        
        
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                
            else:  # Sell
                
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)
        
        elif order.status==order.Canceled:
            self.log('Order Canceled. Price: {}'.format(order.price))
        elif order.status==order.Margin:
            self.log('Order Margin. Price: {}'.format(order.price))
        elif order.status==order.Rejected:
            self.log('Order Rejected. Price: {}'.format(order.price))

        self.order = None

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.enableLog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
    
    
    def notify_timer(self, timer, when, *args, **kwargs):
        if  self.order:
            return
        
        #creating the formula for the current period
        first_rank = {d:eval(self.p.first_filter) for d in self.datas}
        second_rank = {d:eval(self.p.second_filter) for d in self.datas}

        
        first_rank = pd.DataFrame(first_rank, index=['first_filter']).T
        second_rank = pd.DataFrame(second_rank, index=['second_filter']).T
        magic_formula = pd.concat([first_rank, second_rank], axis=1) 
        
        #index | first_filter | second_filter
        #  A   |       a00    |      a01
        #  B   |       a10    |      a11

        magic_formula['first_filter_rank'] = magic_formula['first_filter'].rank(ascending=False, method='first')
        magic_formula['second_filter_rank'] = magic_formula['second_filter'].rank(ascending=False, method='first')
        magic_formula['MAGIC_FORMULA_rank'] = magic_formula['first_filter_rank']+magic_formula['second_filter_rank']
        magic_formula.sort_values('MAGIC_FORMULA_rank', ascending=False, inplace=True)

        #selecting top companies with highest rank
        top_ranks = magic_formula.index[:self.p.stocks_in_portfolio]

        
        #getting companies current parameters (index 0 approach)
        winners_μ = [self.driftAcoes[winner][0] for winner in top_ranks]
        winners_σ = [self.volatAcoes[winner][0] for winner in top_ranks]

        #estimating kelly criterion for current parameters
        winners_w = kelly_criterion().gBm(winners_μ, winners_σ)


        #current positions opened in portfolio
        current_positions = [d for d, pos in self.getpositions().items() if pos]

        
        # remove those no longer top ranked
        # do this first to issue sell orders and free cash
        for d in [d for d in current_positions if d not in top_ranks]:
            self.log('Selling: {}'.format(d._name))
            self.order = self.order_target_percent(d, target=0.0)

        
        #separate companies in which we're going to increase position size
        # vs decrease position size
        reduce_pos = []
        increase_pos = []
        for winner,bet_target in zip(top_ranks, winners_w):
            winner_pos_size = self.getposition(winner, self.broker).size
            if winner_pos_size>bet_target:
                reduce_pos.append((winner, bet_target))
            else:
                increase_pos.append((winner, bet_target))
        
        
        for winner, bet_target in reduce_pos:
            self.log('Reducing position: {} /---/ Rank: {:.4f} '.format(winner._name, rank[winner]))
            self.order_target_percent(winner, target=bet_target)
        
        for winner, bet_target in increase_pos:
            self.log('Increasing position: {} /---/ Rank: {:.4f} '.format(winner._name, rank[winner]))
            self.order_target_percent(winner, target=bet_target)
        
        
    def stop(self):
        self.log('Ending Value %.2f' %
                 (self.broker.getvalue()), doprint=True)
    

def runStrat(in_start, in_end):
    cerebro = bt.Cerebro()
    
    #Set cash value
    cerebro.broker.set_cash(100000)
    
    #Set commission model
    cerebro.broker.setcommission(commission=0)
    
    cerebro.broker.set_checksubmit(False)
    
    files = os.listdir('data')
    for file in files:
        datapath = 'data//'+file
        data = GenericCSVData(
            dataname=datapath,
            fromdate=start,
            todate=end)
        cerebro.adddata(data)
    
    
    
    #Add analyzers
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe')
    
    #Add Strategy    
    cerebro.addstrategy(momentumStrat)
    
    #Run Algo
    results = cerebro.run(maxcpus=1)

    
    #Plot results
    #cerebro.plot()
    
    return results

def analyzeStrat(results, live):
    strat = results[0]
    pyfoliozer = strat.analyzers.getbyname('pyfolio')
    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    
    pf.create_full_tear_sheet(
        returns=returns,
        positions=positions,
        transactions=transactions,
        live_start_date=live,  # This date is sample specific
        estimate_intraday=False
    )

if __name__=='__main__':
    live = '2020-12-31'
    r = runStrat('2009-01-01', '2020-12-31')
    analyzeStrat(r, live)